## Assignment 7 - Sentiment

In [1]:
import tweepy 
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import textblob
import time

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
consumer_key = config.consAPIkey
consumer_secret = config.consAPISecret
access_token = config.accessToken
access_token_secret = config.accessTokenSecret

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [2]:
def textAnalyzer(target_string):
    compound = analyzer.polarity_scores(target_string)["compound"]
    pos = analyzer.polarity_scores(target_string)["pos"]
    neu = analyzer.polarity_scores(target_string)["neu"]
    neg = analyzer.polarity_scores(target_string)["neg"]

    # Print Analysis
    #print("\n\n"+target_string+"\n\n")
    #print("Compound Score: %s" % compound)
    #print("Positive Score: %s" % pos)
    #print("Neutral Score: %s" % neu)
    #print("Negative Score: %s" % neg)
    return {"text":target_string,"compound":compound,"pos":pos,"neu":neu,"neg":neg}

def makePlot(x,y,figname,target):
    ypol = []
    for tweet in y:
        #blob = textblob.TextBlob(tweet)
        VADERAnalysis = textAnalyzer(tweet)
        compound = VADERAnalysis['compound']
        ypol.append(compound)
    plt.plot(x,ypol,'-o')
    plt.xlabel("Tweet #")
    plt.ylabel("Compound Sentiment")
    plt.title(target+" sentiment (compound)")
    plt.savefig(figname)
    plt.close()
    
def resondToTweet(filename,status_id,target,user):
    statusText = "@"+user+ " Hello! Here is your requested analysis of "+target
    api.update_with_media(filename=filename,in_reply_to_status_id_str=status_id,status=statusText)
    
def getOnlyNewTweets(history,newMentions):
    return_list_new = []
    for i in range(len(newMentions)):
        if newMentions[i]['id'] in [item['id'] for item in history]:
            pass
        elif newMentions[i]['text'].split(":")[-1].strip() in [item['text'].split(":")[-1].strip() for item in history]:
            pass
        else:
            return_list_new.append(newMentions[i])
    return return_list_new

def analyzeTweetsAndSend(tweet_dict):
    user = tweet_dict['user']['screen_name'] #User that requested analysis
    id_str = tweet_dict['id_str'] #This is the id of the tweet that requested analysis
    filename = id_str+".png" #This will be the file name that we will use to save the figure of the analysis
    analysis_target = tweet_dict['text'].split(":")[-1].strip() #This is the target who's tweets we will analyze
    tweet_text = []
    for x in range(25):
        # Get all tweets from target user
        public_tweets = api.user_timeline(analysis_target, page=x)    
        # Loop through all tweets and append tweet text to list
        for item in public_tweets:
            tweet_text.append(item['text'])
    tweet_list_enum = [(i-len(tweet_text)+1,text) for i,text in enumerate(tweet_text)]
    x = [item[0] for item in tweet_list_enum]
    y = [item[1] for item in tweet_list_enum]
    makePlot(x,y,filename,analysis_target)
    resondToTweet(filename,id_str,analysis_target,user)


In [3]:
#1. Scan for metions every 5 minutes seconds
tweet_history_list = []
while(True):
    # Do a check on the mentions timeline and see if the contain the keyword 
    # "Sentiment" in the text
    mentions_list = [item for item in api.mentions_timeline() if "sentiment" in item['text'].lower()]
    if(len(mentions_list)>0):
        #1. Check and return only the tweets that are new AND were from accounts NOT previously analyzed. 
        new_tweets_to_analyze = getOnlyNewTweets(tweet_history_list,mentions_list)
        print("we have %d new mentions!" % (len(new_tweets_to_analyze)))
        for i in range(len(new_tweets_to_analyze)):
            analyzeTweetsAndSend(new_tweets_to_analyze[i])
            #2. Add new tweet to history
            tweet_history_list.append(new_tweets_to_analyze[i])
    time.sleep(5*60)

we have 3 new mentions!
we have 0 new mentions!
we have 0 new mentions!
we have 0 new mentions!
we have 0 new mentions!
we have 0 new mentions!
we have 1 new mentions!
we have 0 new mentions!


KeyboardInterrupt: 